In [1]:
import codecs
def read_data(filename):
    with codecs.open(filename, 'r', 'utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

train_data = read_data('../nsm/small_ratings_train.txt')
test_data = read_data('../nsm/small_ratings_test.txt')

print(len(train_data))
print(len(test_data))
print (train_data[0])
print (test_data[0])

10000
100
['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0']
['7898805', '음악이 주가 된, 최고의 음악영화', '1']


In [2]:
%%time
# Open-text 형태소 분석기 import
import MeCab
from mecab_split import mecabsplit
tagger = MeCab.Tagger()

def tokenize(doc):
    return ' '.join(mecabsplit(tagger, doc, False))
    #return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]
#print (train_data[0])
#line = mecabsplit(tagger,train_data[0][1],False)


#train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
train_docs_X = [tokenize(row[1]) for row in train_data]
train_Y = [row[2] for row in train_data]

test_docs_X = [tokenize(row[1]) for row in test_data]
test_Y = [row[2] for row in test_data]


Wall time: 596 ms


In [3]:
print(train_docs_X[0])
print(train_Y[0])
print(train_docs_X[1])
print(train_Y[1])

원작 의 긴장 감 을 제대로 살리 어 내 지 못하 았 다 .
0
액션 이 없 는데 도 재미 있 는 몇 안 되 는 영화
1


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(max_features = 1000).fit(train_docs_X)

train_X = vec.transform(train_docs_X).toarray()
test_X = vec.transform(test_docs_X).toarray()


In [5]:
print(train_X[0])
print(len(train_X))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [6]:
%%time
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(train_X, train_Y)

print("Accuracy on training set: {:.3f}".format(gnb.score(train_X, train_Y)))
print("Accuracy on test set: {:.3f}".format(gnb.score(test_X, test_Y)))

Accuracy on training set: 0.724
Accuracy on test set: 0.720
Wall time: 595 ms


In [7]:
pos = [ [test_docs_X[i], test_Y[i], gnb.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '1' ]
neg = [ [test_docs_X[i], test_Y[i], gnb.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '0' ]
#crr= []
#for i in range(len(test_Y)):
#    if test_Y[i] == '1':
#        crr.append([test_docs_X[i], test_Y[i], gnb_predict([test_X[i]]) ])
#print (crr)
print ('Predicted as Positive:')
for i in pos[:5]:
    print ('입력', i[0])
    print ('정답', i[1])
    print ('출력', i[2])
print ('\nPredicted as Negative:')
for i in neg[:5]:
    print ('입력', i[0])
    print ('정답', i[1])
    print ('출력', i[2])


Predicted as Positive:
입력 음악 이 주 가 되 ᆫ , 최고 의 음악 영화
정답 1
출력 ['1']
입력 괜찮 네요 오랜 만 포켓몬스터 재 ᄆ 밌어요
정답 1
출력 ['1']
입력 청춘 은 아름답 다 그 아름답 ᄆ 은 이성 을 흔들 어 놓 는다 . 찰나 의 아름답 ᄆ 을 잘 포착 하 ᆫ 섬세 하 고 아름답 ᆫ 수채 화 같 은 퀴어 영화 이 다 .
정답 1
출력 ['1']
입력 눈 에 보이 는 반전 이 었 지만 영화 의 흡인 력 은 사라지 지 않 았 다 .
정답 1
출력 ['0']
입력 최고 ! !!!!!!!!!!!!!!!
정답 1
출력 ['1']

Predicted as Negative:
입력 뭣 이 야 이 평점 들 은 . ... 나쁘 지 ㄴ 않 지만 10 점 짜리 는 더더욱 아니 잖아
정답 0
출력 ['1']
입력 지루 하 지 는 않 은데 완전 막장 이 ᄆ . .. 돈 줏 고 보 기 에 는 . ...
정답 0
출력 ['0']
입력 3 D 만 아니 었 어도 별 다섯 개 주 ㅓㅆ 을 터 이 ᆫ데 . . 왜 3 D 로 나오 아서 저 의 심기 를 불편 하 게 않 죠 ? ?
정답 0
출력 ['1']
입력 진정 하 ᆫ 쓰레기
정답 0
출력 ['1']
입력 한국 독립 영화 의 한계 그렇게 아버지 가 되 ᆫ다 와 비교 되 ᄆ
정답 0
출력 ['0']
